## 如何使用函数装饰器

### 问题 

[题目1]斐波那契数列（Fibonacci Sequence），又称黄金分割数列，

指的是这样一个数列：1，1，2，3，5，8，13，21，...

这个数列从第三项开始，每一项都等于前两项之和。求数列第n项。

In [9]:
def fibonacci(n):
    if n <= 1:
        return 1
    return fibonacci(n - 1) + fibonacci(n - 2)

[题目2]一个共有10个台阶的楼梯，从下面走到上面，一次只能迈1-3个台阶，

并且不能后退，走完这个楼梯共有多少种方法。

In [10]:
def climb(n, steps):
    count = 0
    if n == 0:
        count = 1
    elif n > 0:
        for step in steps:
            count += climb(n - step, steps)
    return count

### 测试

In [37]:
fibonacci(30)

1346269

In [13]:
climb(30, [1, 2, 3])

53798080

In [38]:
cache = {}

In [39]:
def fib_with_cache(n):
    if n in cache:
        return cache[n]
    
    if n <= 1:
        return 1
    
    cache[n] = fib_with_cache(n - 1) + fib_with_cache(n - 2)
    
    return cache[n]

In [41]:
%time fibonacci(30)

Wall time: 410 ms


1346269

In [43]:
%time fib_with_cache(30)

Wall time: 0 ns


1346269

### 使用装饰器添加缓存功能

In [56]:
def cache_func(func):
    cache = {}
    def wrap(*args):
        if args not in cache:
            cache[args] = func(*args)
        return cache[args]
    return wrap

In [57]:
@cache_func
def fibonacci(n):
    if n <= 1:
        return 1
    return fibonacci(n - 1) + fibonacci(n - 2)

In [58]:
fibonacci(50)

20365011074

In [59]:
@cache_func
def climb(n, steps):
    count = 0
    if n == 0:
        count = 1
    elif n > 0:
        for step in steps:
            count += climb(n - step, steps)
    return count

In [60]:
climb(10, (1, 2, 3))

274

## 如何为被装饰的函数保存元数据

### 函数的元数据

In [62]:
def f():
    """function f"""
    pass

In [63]:
f.__name__

'f'

In [64]:
f.__doc__

'function f'

In [67]:
f.__module__

'__main__'

In [69]:
f.__defaults__

In [70]:
def f(a, b=1, c=[]):
    print(a, b, c)

In [71]:
f.__defaults__

(1, [])

In [72]:
f.__defaults__[1].append("abc")

In [75]:
f(100)

100 1 ['abc']


In [76]:
f.__closure__

In [77]:
def f():
    a = 2
    return lambda k: a ** k

In [79]:
g = f()

In [80]:
g.__closure__

(<cell at 0x0535F9F0: int object at 0x61096890>,)

In [81]:
c = g.__closure__[0]

In [82]:
c.cell_contents

2

### 装饰器覆盖函数元数据

In [132]:
def mydecorator(func):
    def wrapper(*args, **kwargs):
        """wrapper funcion"""
        print("In wrapper")
        func(*args, **kwargs)
    return wrapper

In [133]:
@mydecorator
def example():
    """example function"""
    print("In example")

In [134]:
print(example.__name__)
print(example.__doc__)

wrapper
wrapper funcion


In [147]:
from functools import wraps, update_wrapper, WRAPPER_ASSIGNMENTS, WRAPPER_UPDATES

In [148]:
def mydecorator(func):
    def wrapper(*args, **kwargs):
        """wrapper funcion"""
        print("In wrapper")
        func(*args, **kwargs)
    update_wrapper(wrapper, func, ("__name__", "__doc__"), ("__dict__",))
    return wrapper

In [149]:
@mydecorator
def example():
    """example function"""
    print("In example")

In [151]:
print(example.__name__)
print(example.__doc__)
print(WRAPPER_ASSIGNMENTS)
print(WRAPPER_UPDATES)

example
example function
('__module__', '__name__', '__qualname__', '__doc__', '__annotations__')
('__dict__',)


In [152]:
def mydecorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        """wrapper funcion"""
        print("In wrapper")
        func(*args, **kwargs)
    return wrapper

In [153]:
@mydecorator
def example():
    """example function"""
    print("In example")

In [154]:
print(example.__name__)
print(example.__doc__)

example
example function


## 如何定义带参数的装饰器

In [157]:
from inspect import signature

In [168]:
def typeassert(*ty_args, **ty_kwargs):
    def decorator(func):
        sig = signature(func)
        bind_types = sig.bind_partial(*ty_args, **ty_kwargs).arguments
        def wrapper(*args, **kwargs):
            for name, obj in sig.bind(*args, **kwargs).arguments.items():
                if name in bind_types:
                    if not isinstance(obj, bind_types[name]):
                        raise TypeError(f"{obj} must be {bind_types[name]}")
            return func(*args, **kwargs)
        return wrapper
    return decorator

In [169]:
@typeassert(int, int)
def f(a, b):
    print(a, b)

In [170]:
f(1, 2)

1 2


In [171]:
f(1, 'a')

TypeError: a must be <class 'int'>

In [172]:
@typeassert(b=int, c=list)
def f1(a, b=1, c=[]):
    print(a, b, c)

In [173]:
f1('a', 5, ())

TypeError: () must be <class 'list'>

## 如何实现属性可修改的装饰器

In [232]:
import time
import logging
from functools import wraps
from random import randint

In [233]:
def warn(timeout):
    def decorator(func):
        def wrapper(*args, **kwargs):
            start = time.time()
            res = func(*args, **kwargs)
            used = time.time() - start
            if used > timeout:
                msg = f"{func.__name__}: {used}  >  {timeout}"
                logging.warn(msg)
            return res
        return wrapper
    return decorator

In [234]:
@warn(1.5)
def test():
    print("In test")
    
    while randint(0, 1):
        time.sleep(0.5)

In [235]:
for i in range(30):
    test()

In test
In test
In test


c:\users\xaut\.virtualenvs\learn-python-ckwvhrp2\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: The 'warn' function is deprecated, use 'warning' instead
  if __name__ == '__main__':


In test
In test
In test
In test
In test
In test
In test
In test
In test
In test
In test
In test
In test
In test


In test
In test
In test
In test
In test
In test
In test
In test
In test


In test
In test
In test
In test


In [267]:
def warn(timeout):
    def decorator(func):
        def wrapper(*args, **kwargs):
            start = time.time()
            res = func(*args, **kwargs)
            used = time.time() - start
            if used > timeout:
                msg = f"{func.__name__}: {used}  >  {timeout}"
                logging.warn(msg)
            return res
        def setTimeout(n):
            nonlocal timeout
            timeout = n
        wrapper.setTimeout = setTimeout
        return wrapper
    return decorator

In [272]:
# python2
def warn(timeout):
    timeout = [timeout]
    def decorator(func):
        def wrapper(*args, **kwargs):
            start = time.time()
            res = func(*args, **kwargs)
            used = time.time() - start
            if used > timeout[0]:
                msg = f"{func.__name__}: {used}  >  {timeout[0]}"
                logging.warn(msg)
            return res
        def setTimeout(n):
            # nonlocal timeout
            timeout[0] = n
        wrapper.setTimeout = setTimeout
        return wrapper
    return decorator

In [273]:
@warn(1.5)
def test():
    print("In test")
    
    while randint(0, 1):
        time.sleep(0.5)

In [274]:
for i in range(30):
    test()
    
test.setTimeout(1)
for i in range(30):
    test()

In test
In test
In test
In test
In test
In test
In test
In test
In test
In test
In test
In test
In test
In test
In test
In test
In test


c:\users\xaut\.virtualenvs\learn-python-ckwvhrp2\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: The 'warn' function is deprecated, use 'warning' instead
  # This is added back by InteractiveShellApp.init_path()


In test
In test
In test


In test
In test
In test
In test
In test


In test
In test
In test
In test
In test


In test


## 如何在类中实现装饰器

In [275]:
import logging
from time import localtime, time, strftime, sleep

In [277]:
import logging
from time import localtime, time, strftime, sleep

class CallingInfo(object):
    def __init__(self, name):
        log = logging.getLogger(name)
        log.setLevel(logging.INFO)
        fh = logging.FileHandler(name + '.log')
        log.addHandler(fh)
        log.info('Start'.center(50, '-'))
        self.log = log
        self.formatter = '%(func)s -> [%(time)s - %(used)s - %(ncalls)s]'

    def info(self, func):
        def wrapper(*args, **kargs):
            wrapper.ncalls += 1
            lt = localtime()
            start = time()
            res = func(*args, **kargs)
            used = time() - start
            info = {}
            info['func'] = func.__name__
            info['time'] = strftime('%x %X', lt)
            info['used'] = used
            info['ncalls'] = wrapper.ncalls
            msg = self.formatter % info
            self.log.info(msg)
            return res
        wrapper.ncalls = 0
        return wrapper

    def setFormatter(self, formatter):
        self.formatter = formatter

    def turnOn(self):
        self.log.setLevel(logging.INFO)

    def turnOff(self):
        self.log.setLevel(logging.WARN)


cinfo1 = CallingInfo('mylog1')
cinfo2 = CallingInfo('mylog2')

cinfo1.setFormatter('%(func)s -> [%(time)s - %(ncalls)s]')
cinfo2.turnOff()

@cinfo1.info
def f():
    print ('in f')

@cinfo1.info
def g():
    print ('in g')

@cinfo2.info
def h():
    print ('in h')

from random import choice

for _ in range(50):
    choice([f, g, h])()
    sleep(choice([0.5, 1, 1.5]))




INFO:mylog1:----------------------Start-----------------------
INFO:mylog2:----------------------Start-----------------------


in h
in h


INFO:mylog1:f -> [06/18/19 22:02:23 - 1]


in f


INFO:mylog1:f -> [06/18/19 22:02:24 - 2]


...

in f
in h


INFO:mylog1:g -> [06/18/19 22:03:05 - 10]


in g


INFO:mylog1:g -> [06/18/19 22:03:06 - 11]


in g


## 如何 ？？？

In [283]:
from types import MethodType

In [284]:
class NCalls(object):
    def __init__(self, func):
        self.func = func
        self._ncalls = 0

    def __call__(self, *args, **kargs):
        print('in __call__')
        self._ncalls += 1
        return self.func(*args, **kargs)

    def __get__(self, instance, cls):
        return MethodType(self, instance, cls)

    def ncalls(self):
        return self._ncalls

    def reset(self):
        self._ncalls = 0

In [295]:
@NCalls
def f():
    print('in f')

# f = NCalls(f)
# '''
f()
f()
f()

print(f.ncalls())
# '''

in __call__
in f
in __call__
in f
in __call__
in f
3


In [287]:
class A(object):
    @NCalls
    def g(self):
        print('in A::g', self)

    def h(self):
        print('in A::h', self)

a = A()
a.g()
a.g()
a.g()
a.g()
print(a.g.ncalls())

TypeError: method expected 2 arguments, got 3